<!--NOTEBOOK_HEADER-->
*This notebook contains material from [PyRosetta](https://RosettaCommons.github.io/PyRosetta);
content is available [on Github](https://github.com/RosettaCommons/PyRosetta.notebooks.git).*

# Working With Symmetry
Keywords: 

## Overview
Symmetry is an important concept to learn when working with biomolecules.  When a protein is crystalized, it is in the precense of its symmetrical neighbors - which can be important if testing particular protocols or using crystal density for refinement or full structure building.

Symmetry can also be useful for designing symmetrical structures or large repeating meta-proteins like protein cages. 

### Symmetry In Rosetta
So why do we care if our protein is symmetrical or not when it comes to Rosetta?  Each residue and atom that is loaded into Rosetta takes time to both load, and time to score.  Since scoring can happen thousands of times - even in a short protocol, anything we can do to speed this up becomes important. The most expensive operation in Rosetta is minimization, and by using symmetry - we can reduce the minimization time exponentially by minimizing a single copy instead of ALL copies.  We will get into the details about how this works below.

When we use symmetry in Rosetta - we are basically telling rosetta that the symmetrical partners are 'special'.  Upon setting up symmety in Rosetta, Rosetta will replace the `Conformation` within the pose with a **Symmetrical** version, called the `SymmetricConformation`.  If you know anything about classes, this `SymmetricConformation` is derived from the actual `Conformation` object, but contains extra information about the pose and some functions are replaced. 

### Symmetric Scoring and Moving
Ok, so now lets assume that we have our symmetric pose.  Now what?  Well, the symmetric copies are all tied to their real counterparts.  Once you move a chain, residue, or atom by packing or minimization, the symmetric copies of that residue are all moved in the same way. 

Cool.  But what about scoring?  Scoring works very similarly - instead of scoring each and every residue in our pose, Rosetta will score just our assymetric unit, and multiply that out to the number of symmetric copies we have.  Intelligently, Rosetta will also figure out the symmetric interfaces that arise from the interactions of our assymetric unit to the symmetric copies and score them appropriately.

### Symmetry-aware movers
Most of our common movers are symmetry-aware.  At one point there were different symmetric and non-symmetric versions of particular code, such as MinMover and PackRotamersMover.  Now though, Rosetta will automatically use the pose to figure out what needs to be done. You should seek original documentation (and contact the author if not explicit) to make sure that an uncommon protocol you are using is symmetry-aware.

## Documentation
More information on RosettaSymmetry can be found in the following places:
- https://www.rosettacommons.org/docs/latest/rosetta_basics/structural_concepts/symmetry
- https://www.rosettacommons.org/demos/latest/tutorials/Symmetry/Symmetry
- https://www.rosettacommons.org/docs/latest/application_documentation/utilities/make-symmdef-file
- https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/SetupForSymmetryMover
- https://www.rosettacommons.org/docs/latest/scripting_documentation/RosettaScripts/Movers/movers_pages/ExtractAsymmetricUnitMover


In [ ]:
# Notebook setup
!pip install pyrosettacolabsetup
import pyrosettacolabsetup
pyrosettacolabsetup.setup()
print ("Notebook is set for PyRosetta use in Colab.  Have fun!")

Here, we will use a few specific options. The first three options make Rosetta a bit more robust to input structures.  The `-load_PDB_components` cannot be used with glycans, unfortunately, and our structure has a few very important glycans.  Finally, we load a bunch of glycan-specific options, which we will cover in the next tutorial.

In [7]:
from pyrosetta import *
from pyrosetta.rosetta import *
from pyrosetta.teaching import *
import os

init('-ignore_unrecognized_res -load_PDB_components false -ignore_zero_occupancy false @inputs/glycan_flags')

PyRosetta-4 2019 [Rosetta PyRosetta4.Release.python36.mac 2019.33+release.1e60c63beb532fd475f0f704d68d462b8af2a977 2019-08-09T15:19:57] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Reading fconfig.../Users/jadolfbr/.rosetta/flags/common
core.init: 
core.init: 
core.init: Rosetta version: PyRosetta4.Release.python36.mac r230 2019.33+release.1e60c63beb5 1e60c63beb532fd475f0f704d68d462b8af2a977 http://www.pyrosetta.org 2019-08-09T15:19:57
core.init: command: PyRosetta -ignore_unrecognized_res -load_PDB_components false -ignore_zero_occupancy false @inputs/glycan_flags -database /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pyrosetta-2019.33+release.1e60c63beb5-py3.6-macosx-10.6-intel.egg/pyrosetta/database
basic.random.init_random_generator: 'RNG device' seed mode, using '/dev/urandom'

## Creating a SymDef file

Here, we will start with how to create a basic symdef file for cyrstal symmetry.  Note that there are ways to do this without a symdef file, but these do not currently work for glycan structures, which we will be using here. 

The `make_symdef_file.pl` file is within Rosetta3. To use it, you will need to download and licence Rosetta3.  The code is in the `Rosetta/main/src/apps/public` directory. In the interest of reducing code drift, this file is NOT included in the tutorial directory as we may then have version drift.  

If you have done this, we can use the following command to create the symdef file.  Here, the radius of symmetrical partners is 12A, which is certainly fairly large, but produces a very well represented crystal.

In [1]:
base_cmd = "cd inputs && make_symmdef_file.pl -r 12 -m CRYST -p  {pdb}.pdb > {pdb}_crys.symm && cd -"

Use this base command and the `os.system(cmd)` function to run the code or use the provided symdef file. 

In [4]:
os.system('cp inputs/1jnd_crys.symm .')

0

Take a look at the symmetrized structure in pymol (`inputs/1jnd_symm.pdb`).  What would happen if we increased the radius to 24 instead of 12?

## Running Rosetta with Symmetry

Here, we will run a basic Rosetta protocol with symmetry.  There are much more complicated things you can do with symmetry, but for now, we just want to symmetrically pack the protein.  Please see the docs for more on symmetry.  The full Rosetta C++ tutorial for symmetry is a great place to go from here: - https://www.rosettacommons.org/demos/latest/tutorials/Symmetry/Symmetry 

In [8]:
p = pose_from_pdb('inputs/1jnd_refined.pdb.gz')

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 1251 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.24338 seconds.
core.import_pose.import_pose: File 'inputs/1jnd_refined.pdb.gz' automatically determined to be of type PDB
core.io.pdb.pdb_reader: Parsing 411 .pdb records with unknown format to search for Rosetta-specific comments.
core.io.util: Automatic glycan connection is activated.
core.io.util: Start reordering residues.
core.io.util: Corrected glycan residue order (internal numbering): [401, 402, 403, 404]
core.io.util: 
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] Glc401 has an unfavorable ring conformation; the coordinates for this input structure may have been poorly assigned.
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] Glc402 has an unfavorable ring conformation; the coordinates for this input structure may have been poorly assigned.
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WAR